In [7]:
import cv2
import numpy as np
import random
import torch
from PIL import Image
import json
import os

random.seed(42)

In [8]:
def resize_image(image_path, resized_height=1600, resized_width=1200):
    image = cv2.imread(image_path)
    resized_image = cv2.resize(image, (resized_width, resized_height))
    return resized_image

In [3]:
def add_random_offset(point, max_offset):
        return point + random.randint(-max_offset, max_offset)

In [4]:
def random_rgb_color():
    r = random.randint(0, 255)
    g = random.randint(0, 255)
    b = random.randint(0, 255)
    return (r, g, b)

In [10]:
def draw_broken_box_with_subboxes(background, rect_start, width, height, thickness, delta, l1, l2, max_offset=10, color=(0, 255, 0), opacity=0.5):
    """
    Draw a rectangular box broken into smaller subboxes with slight irregularities in width and height.
    
    Parameters:
        background (OpenCV Image): OpenCV Matrix representing the (already resized) image.
        rect_start (tuple): Starting coordinate (x, y) of the rectangle (top-left corner).
        width (int): Width of the overall rectangle.
        height (int): Height of the overall rectangle.
        thickness (int): Height of each smaller subbox.
        delta (int): Reduction in height for each smaller subbox.
        max_offset (int): Maximum random offset for width irregularity (in pixels).
        color (tuple): Color of the rectangle (BGR format).
        opacity (float): Opacity of the shape (0: fully transparent, 1: fully opaque).
        
    Returns:
        result_img (ndarray): Image with the broken box and irregular subboxes.
        bbox (dict): Metadata of the bounding box for YOLO.
    """
    
    # Create an overlay image (same size as the background)
    overlay = background.copy()

    # Define the top-left starting point of the rectangle
    x, y = rect_start
    
    # Number of subboxes along the height
    num_subboxes = height // thickness
    
    # Initialize the bounding box metadata
    bbox = {
        "x_center": (x + width / 2) / background.shape[1],  # Normalize x center
        "y_center": (y + height / 2) / background.shape[0],  # Normalize y center
        "width": width / background.shape[1],                # Normalize width
        "height": height / background.shape[0],              # Normalize height
        "class": 0                                            # Assuming class 0 for YOLO
    }

    # Iterate over each subbox to create the broken box pattern
    for i in range(num_subboxes):
        # Starting Y position of the current subbox
        subbox_start_y = y + i * thickness
        # Ending Y position of the current subbox (apply delta to reduce height)
        subbox_end_y = subbox_start_y + thickness - delta
            
        # Apply random width variation
        random_width_start = add_random_offset(x, max_offset)
        random_width_end = add_random_offset(x + width, max_offset)

        if(i == 0):
            random_width_start += l1
        elif(i == num_subboxes-1):
            random_width_end -= l2
        
        # Define the points of the subbox (irregular rectangle)
        pts = np.array([
            [add_random_offset(random_width_start, max_offset), add_random_offset(subbox_start_y, max_offset)],  # Top-left
            [add_random_offset(random_width_end, max_offset), add_random_offset(subbox_start_y, max_offset)],    # Top-right
            [add_random_offset(random_width_end, max_offset), add_random_offset(subbox_end_y, max_offset)],      # Bottom-right
            [add_random_offset(random_width_start, max_offset), add_random_offset(subbox_end_y, max_offset)]     # Bottom-left
        ], np.int32)
        
        # Draw the subbox on the overlay
        cv2.fillPoly(overlay, [pts], color)
    
    # Perform alpha blending (opacity) between the background and overlay
    result_img = cv2.addWeighted(overlay, opacity, background, 1 - opacity, 0)

    return result_img, bbox

In [11]:
training_custom_metadata = './background_images/train/metadata_train.json'

with open(training_custom_metadata, 'r') as f:
    metadata = json.load(f)   

In [20]:
training_dataset_path = "./background_images/train"
training_bbox_data = './generated_dataset/train/bboxes_train.json'
generated_train_dataset_path = "./generated_dataset/train"
resized_img_height = 1600
resized_img_width = 1200
variations_per_image = 5

for image in metadata["images"]:
    image_filename = image["filename"]
    image_path = os.path.join(training_dataset_path, image_filename)
    bg_img = resize_image(image_path, resized_img_height, resized_img_width)

    with open(training_bbox_data, "a") as f:
        for box_num, box in enumerate(image["boxes"]):
            # Get initial box parameters
            original_rect_start = box["rect_start"][:]
            width = box["width"]
            height = box["height"]
            thickness = box["thickness"]
            delta = box["line_spacing"]
            max_lines = height // thickness
            
            for variation_num in range(variations_per_image):  
                # Set random starting position within the rectangle bounds
                random_factor = thickness * random.randint(0, 3)
                variation_rect_start = original_rect_start[:]
                variation_rect_start[1] += random_factor
                adjusted_height = height - random_factor  # Adjust height
                
                # Ensure the rectangle is within the image bounds
                variation_rect_start[1] = min(variation_rect_start[1], resized_img_height - adjusted_height)
                
                l1, l2 = random.randint(0, 100), random.randint(0, 100)
                max_offset = random.randint(0, 10)
                color = random_rgb_color()
                opacity = random.uniform(0.05, 0.6)

                print(f"rect_start = {rect_start}\n width = {width} \n height = {adjusted_height}")
                # Draw the adjusted rectangle
                result_image, bbox_metadata = draw_broken_box_with_subboxes(
                    bg_img, variation_rect_start, width, adjusted_height, thickness, delta, l1, l2, max_offset, color, opacity
                )
                print(f"\n")

                # Save each variation with a unique filename
                filename, extension = os.path.splitext(image_filename)
                result_image_name = f"{filename}_box_{box_num}_variation_{variation_num}{extension}"
                result_image_filepath = os.path.join(generated_train_dataset_path, result_image_name)
            
                cv2.imwrite(result_image_filepath, result_image)
                json.dump(bbox_metadata, f)  # Append the JSON object (box) to the file
                f.write("\n")  # Write each box on a new line

rect_start = [215, 1835]
 width = 646 
 height = 38


rect_start = [215, 1835]
 width = 646 
 height = 10


rect_start = [215, 1835]
 width = 646 
 height = 10


rect_start = [215, 1835]
 width = 646 
 height = -18


rect_start = [215, 1835]
 width = 646 
 height = 10


rect_start = [215, 1835]
 width = 656 
 height = 381


rect_start = [215, 1835]
 width = 656 
 height = 453


rect_start = [215, 1835]
 width = 656 
 height = 381


rect_start = [215, 1835]
 width = 656 
 height = 405


rect_start = [215, 1835]
 width = 656 
 height = 429


